In [1]:
import os
import train_model
from aws_helpers import create_repository, setup_elb, create_ecs_service, deregister_old_taskdefinitions
from keras_to_tfserving import convert_keras_to_tf_model

/home/maxber/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Reqs.
This code requires a couple of things:
- Installed requirements
- Docker (https://docs.docker.com/install/linux/docker-ce/ubuntu/)
- installed & configured AWS CLI (https://aws.amazon.com/cli/)
- AWS Elastic Load Balancer setup as an Application Load Balancer with a listener on 443
- AWS Elastic Container Srevice cluster setup and configured

In [3]:
version = 1
service_path = 'services.dev.sellpy.net'
service = "stockholm-ai"
cluster = "microservices"
load_balancer_name = "sellpy-services"

In [4]:
model_name = "stockholm_ai_mnist.m"
!ls
if model_name not in os.listdir("."):
    train_model.output_model(model_name, epochs=1)

aws_helpers.py	       __pycache__	  server.py
docker-compose.yml     README.md	  stockholm_ai_mnist.m
Dockerfile	       requirements.txt   task_definition_2.json
install.sh	       run_tfserver.sh	  train_model.py
keras_to_tfserving.py  Sample Code.ipynb
models		       server.conf


In [39]:
# note that the model path is referred to in server.conf
# TODO: Make this repeatable in notebook
convert_keras_to_tf_model(model_name=model_name,
                          model_path="models/stockholm-ai",
                          sequential=True)

Loading Sequential model, 
                 specify sequential=False to load functional model
inputs {
  key: "input_0"
  value {
    name: "conv2d_1_input_1:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 28
      }
      dim {
        size: 28
      }
      dim {
        size: 1
      }
    }
  }
}
outputs {
  key: "output_0"
  value {
    name: "dense_2_1/Softmax:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 10
      }
    }
  }
}
method_name: "tensorflow/serving/predict"

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'models/stockholm-ai/0/saved_model.pb'


In [40]:
# Create an image repository for storing docker files.
create_repository(service)

!eval $( aws ecr get-login --no-include-email)
!docker build -t temp_image .
!docker tag temp_image 966836717103.dkr.ecr.eu-west-1.amazonaws.com/stockholm-ai:dev
!docker push 966836717103.dkr.ecr.eu-west-1.amazonaws.com/stockholm-ai:dev
!aws ecs register-task-definition --cli-input-json file://task_definition_2.json --region eu-west-1


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Login Succeeded
Sending build context to Docker daemon  33.92MB
Step 1/10 : FROM ubuntu:16.04
 ---> f975c5035748
Step 2/10 : RUN apt-get update && apt-get install -y         build-essential         curl         git         &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 1b8db2213203
Step 3/10 : ADD install.sh /tmp/install.sh
 ---> Using cache
 ---> 2f8661714cd5
Step 4/10 : RUN sh -e /tmp/install.sh
 ---> Using cache
 ---> 705ffe62372f
Step 5/10 : COPY requirements.txt /tmp/
 ---> Using cache
 ---> c5271e8a6b4e
Step 6/10 : RUN pip3 install -r /tmp/requirements.txt
 ---> Using cache
 ---> 3401bd12a327
Step 7/10 : RUN add-apt-repository ppa:ubuntu-toolchain-r/test -y
 ---> Using cache
 ---> 2bfd47cd6b86
Step 8/10 : RUN apt-get update -y && apt-get upgrade -y && apt-get dist-upgrade -y
 ---> Using cache
 ---> 853316a548c1
Step 9/10 : WORKDIR /app
 ---> Using cache
 ---> 9764e2987889
Ste

In [29]:
target_group_arn = setup_elb(
    service=service,
    version=version,
    load_balancer_name=load_balancer_name,
    service_path=service_path
)

In [46]:
# Create / Update a task definition.
create_ecs_service(
    cluster=cluster,
    service=service,
    task_definition_family=service,
    target_group_arn=target_group_arn,
    service_count=1
)

stockholm-ai:13


In [70]:
def deregister_old_taskdefinitions(service):
    ecs_client = boto3.client('ecs')

    task_defs = ecs_client.list_task_definitions(
        familyPrefix=service,
        status='ACTIVE',
        sort='ASC',
        maxResults=100
    )
    for revision in task_defs["taskDefinitionArns"][0:-1]:
        print()
        ecs_client.deregister_task_definition(
            taskDefinition=revision.split("/")[-1])


In [71]:
# Update existing services
!aws ecs register-task-definition --cli-input-json file://task_definition_2.json --region eu-west-1
!aws ecs update-service --service stockholm-ai --task-definition stockholm-ai --region eu-west-1  --cluster microservices
deregister_old_taskdefinitions(service)

{
    "taskDefinition": {
        "taskDefinitionArn": "arn:aws:ecs:eu-west-1:966836717103:task-definition/stockholm-ai:15",
        "containerDefinitions": [
            {
                "name": "stockholm-ai",
                "image": "966836717103.dkr.ecr.eu-west-1.amazonaws.com/stockholm-ai:dev",
                "cpu": 0,
                "memoryReservation": 300,
                "links": [
                    "serve_tensorflow_sthlm_ai:serve_tensorflow"
                ],
                "portMappings": [
                    {
                        "containerPort": 8080,
                        "hostPort": 1912,
                        "protocol": "tcp"
                    }
                ],
                "essential": true,
                "command": [
                    "python3",
                    "server.py"
                ],
                "environment": [
                    {
                        "name": "MNIST_SECRET_KEY",
                        "value": "sto